In [ ]:
# Importing the required packages and libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model

# Loading the clean data as CSV 
df = pd.read_csv("E:\IA\Machine Learning_Manuscript\Refine data\Test Folder\Test_2.csv")  

# Independent variables/major drivers of deforestation
X = df[['Population Growth', 'Livelihood activities','Forest Fire',	'Fuelwood Collection', 'Inadequate Education', 'Poor Forest Management',
        'Illegal Logging',	'Distance from Forest'
]]

# Dependent variable
y = df['Deforestation']

# Adding constant
X = sm.add_constant(X)

# Fitting logistic regression model model
logit_model = sm.Logit(y, X)
result = logit_model.fit()

print(result.summary())
df['Deforestation'].value_counts()


In [ ]:
# Computing Odds Ratios (Exp(B)) as it is missing in previous model:
# X = independent variables (with constant), y = dependent binary variable
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Get coefficients
params = result.params

# Computing exp(B)
odds_ratios = np.exp(params)

print("Odds Ratios (Exp(B)):\n")
print(odds_ratios)

# Getting confidence intervals
conf = result.conf_int()
conf['OR'] = odds_ratios
conf.columns = ['2.5%', '97.5%', 'OR']

# Adding p-values
conf['p-value'] = result.pvalues

# Adding B (coefficients) =
conf['B'] = result.params

print(conf)


In [ ]:
# Combining the results of both models in the table
logit_model = sm.Logit(y, X)
result = logit_model.fit()

# Coefficients and stats
params = result.params
conf = result.conf_int()
conf.columns = ['2.5%', '97.5%']
odds_ratios = np.exp(params)

# Combine everything
summary_table = pd.DataFrame({
    'B': params,
    'SE': result.bse,
    'z-value': result.tvalues,
    'p-value': result.pvalues,
    '95% CI Lower': conf['2.5%'],
    '95% CI Upper': conf['97.5%'],
    'Exp(B)': odds_ratios
})

summary_table = summary_table.round(3)

# Exportting to Excel and display
summary_table.to_excel('logit_summary.xlsx')
summary_table


In [ ]:
#Calculating the Variance Inflation Factor (VF)
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df.drop(columns='Deforestation')  # independent variables
X = add_constant(X)
vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)


In [ ]:
#Performance Matrices for Machine Learning Models

# 1. Importing the required Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    roc_curve, auc
)

# 2. Spliting the Data 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Standardize for SVM & Logistic
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Defining Machine Learning Models 
log_model = LogisticRegression()
rf_model = RandomForestClassifier(random_state=42)
svm_model = SVC(probability=True, kernel='linear', random_state=42)

#5. Training Models 
log_model.fit(X_train_scaled, y_train)
rf_model.fit(X_train, y_train)
svm_model.fit(X_train_scaled, y_train)

# 6. Predicting Probabilities 
y_log_proba = log_model.predict_proba(X_test_scaled)[:, 1]
y_rf_proba = rf_model.predict_proba(X_test)[:, 1]
y_svm_proba = svm_model.predict_proba(X_test_scaled)[:, 1]

# 7. Predicting Classes ===
y_log_pred = log_model.predict(X_test_scaled)
y_rf_pred = rf_model.predict(X_test)
y_svm_pred = svm_model.predict(X_test_scaled)

# === 8. Evaluating Machine Learning Models ===
models = {
    'Logistic Regression': (y_test, y_log_pred, y_log_proba),
    'Random Forest': (y_test, y_rf_pred, y_rf_proba),
    'SVM': (y_test, y_svm_pred, y_svm_proba)
}

for name, (yt, yp, yp_prob) in models.items():
    print(f"\n=== {name} ===")
    print("Accuracy:", accuracy_score(yt, yp))
    print("Classification Report:\n", classification_report(yt, yp))



In [ ]:
# Plotting ROC Curves for AUC values for all three Machine Learning Models  
# Logestic Regression
# Random Forest
# Supoort Vector Machine
fpr_lr, tpr_lr, _ = roc_curve(y_test, log_probs)
fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_probs)
fpr_svm, tpr_svm, _ = roc_curve(y_test, svm_probs)

# AUC scores
auc_lr = auc(fpr_lr, tpr_lr)
auc_rf = auc(fpr_rf, tpr_rf)
auc_svm = auc(fpr_svm, tpr_svm)

# Plotting the ROC Curves
plt.figure(figsize=(6,4))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic (AUC = {auc_lr:.2f})')
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {auc_rf:.2f})')
plt.plot(fpr_svm, tpr_svm, label=f'SVM (AUC = {auc_svm:.2f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (All Models)')
plt.legend(loc='lower right')
plt.grid()
plt.savefig("roc_combined_models.png", dpi=300)
plt.show()


In [ ]:
# Confusion Matrices for all three Machine Learning Models 
# Update below according to actual column names in the CSV file
feature_cols = ['Population Growth', 'Livelihood activities', 'Forest Fire', 'Fuelwood Collection', 
                'Inadequate Education', 'Poor Forest Management', 'Illegal Logging', 'Distance from Forest']
target_col = 'Deforestation'

# 1. Preparing the data
X = df[feature_cols]
y = df[target_col]

# 2. Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Initializing the ML models
log_reg = LogisticRegression(max_iter=1000)
rf_model = RandomForestClassifier(random_state=42)
svm_model = SVC(probability=True, random_state=42)

# 4. Training ML models
log_reg.fit(X_train, y_train)
rf_model.fit(X_train, y_train)
svm_model.fit(X_train, y_train)

# 5. Predictting the ML Models 
log_pred = log_reg.predict(X_test)
rf_pred = rf_model.predict(X_test)
svm_pred = svm_model.predict(X_test)

# 6. Creating confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

for ax, model_name, y_pred in zip(
    axes,
    ['Logistic Regression', 'Random Forest', 'SVM'],
    [log_pred, rf_pred, svm_pred]
):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=ax, values_format='d')
    ax.set_title(f'{model_name}')

plt.tight_layout()
plt.savefig("combined_confusion_matrices.png", dpi=300)
plt.show()

In [ ]:
# Changing the color patterns for already fitted models a

fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Logistic Regression
ConfusionMatrixDisplay.from_estimator(log_reg, X_test, y_test, ax=axes[0], cmap='Blues')
axes[0].set_title('Logistic Regression')

# Random Forest
ConfusionMatrixDisplay.from_estimator(rf_model, X_test, y_test, ax=axes[1], cmap='Greens')
axes[1].set_title('Random Forest')

# SVM
ConfusionMatrixDisplay.from_estimator(svm_model, X_test, y_test, ax=axes[2], cmap='Oranges')
axes[2].set_title('Support Vector Machine')

plt.tight_layout()  # To reduces spacing between subplots
plt.savefig("Combined_Confusion_Matrix.png", dpi=500)
plt.show()


In [ ]:
#Cross Validation of AUC scores to check over fitting the models 

# Defining the ML models
lr = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Defining scoring metric
auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

# Cross-validated AUC
lr_auc_scores = cross_val_score(lr, X, y, cv=5, scoring=auc_scorer)
rf_auc_scores = cross_val_score(rf, X, y, cv=5, scoring=auc_scorer)

print("Logistic Regression AUC Scores:", lr_auc_scores)
print("Mean AUC (LR):", np.mean(lr_auc_scores))
print("\nRandom Forest AUC Scores:", rf_auc_scores)
print("Mean AUC (RF):", np.mean(rf_auc_scores))

svc = SVC(kernel='linear', probability=True)
scores = cross_val_score(svc, X, y, cv=5, scoring='roc_auc')
print(f"SVM AUC scores: {scores}")
print(f"Mean AUC: {scores.mean():.2f}")

In [ ]:
#Plotting Cross Validation AUC scores to check over fitting the models 
# Defining model names and their cross-validated AUC scores
models = ['Logistic Regression', 'Random Forest', 'SVM']
auc_scores = {
    'Logistic Regression': [0.7667, 0.7511, 0.7778, 0.8733, 0.8865],
    'Random Forest': [0.9694, 0.9267, 0.9689, 0.9272, 0.9922],
    'SVM': [0.7478, 0.7433, 0.7956, 0.8722, 0.8654]
}

# Calculating mean AUCs
mean_aucs = [np.mean(auc_scores[model]) for model in models]

# Plotting the AUC scores
plt.figure(figsize=(7, 4))
for i, model in enumerate(models):
    plt.plot(range(1, 6), auc_scores[model], marker='o', label=f'{model} (Mean AUC: {mean_aucs[i]:.3f})')

plt.title('Cross-Validated AUC Scores for ML Models')
plt.xlabel('Fold Number')
plt.ylabel('AUC Score')
plt.ylim(0.70, 1.00)
plt.xticks(range(1, 6))
plt.grid(True)
plt.legend()
plt.tight_layout()

# Saving the figure 
plt.savefig("ml_model_auc_comparison.png", dpi=400)
plt.show()


In [ ]:
# Creating Feature Importance Chart for Random Forest and Support Vector Machine 

#1: Splitting the data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Removing the 'const' 
if 'const' in X_train.columns:
    X_train = X_train.drop(columns=['const'])
    X_test = X_test.drop(columns=['const'])

#Training the Models ---
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

svm = SVC(kernel='linear', probability=True, random_state=42)
svm.fit(X_train, y_train)

# Getting Feature Importances 
# For Random Forest
rf_importances = pd.Series(rf.feature_importances_, index=X_train.columns)

# For SVM Permutation Importance
svm_perm = permutation_importance(svm, X_test, y_test, n_repeats=30, random_state=42)
svm_importances = pd.Series(svm_perm.importances_mean, index=X_train.columns)

#Creating Combined Table 
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'RF_Importance': rf_importances.values,
    'SVM_Importance': svm_importances.values
}).sort_values(by='RF_Importance', ascending=False)

print("\n=== Combined Feature Importance Table ===")
print(importance_df)


# Plotting the Side-by-Side Bar Chart ---
fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

# Plot for RF
axes[0].barh(importance_df['Feature'], importance_df['RF_Importance'], color='green')
axes[0].set_title("Random Forest Importance")
axes[0].set_xlabel("Importance")

# Plot for SVM
axes[1].barh(importance_df['Feature'], importance_df['SVM_Importance'], color='purple')
axes[1].set_title("SVM Permutation Importance")
axes[1].set_xlabel("Importance")

plt.tight_layout()
plt.savefig("Feature_Importance_Comparison_RF_SVM.png")
plt.show()


In [ ]:
pip install shap scikit-learn matplotlib pandas


In [ ]:
# Plotting Predict probabilities Chart for Machine Learning Models 
# Predict probabilities
log_probs = log_reg.predict_proba(X_test)[:, 1]
rf_probs = rf_model.predict_proba(X_test)[:, 1]
svm_probs = svm_model.predict_proba(X_test)[:, 1]

# Sortting by predicted probabilities for cleaner plots
sorted_idx = np.argsort(log_probs)

fig, axes = plt.subplots(1, 3, figsize=(14, 4), sharey=True)

# For Logistic Regression
axes[0].plot(log_probs[sorted_idx], label='Predicted', color='blue')
axes[0].scatter(range(len(y_test)), y_test.values[sorted_idx], label='Actual', color='red', s=10)
axes[0].set_title('Logistic Regression')
axes[0].set_xlabel('Sorted Observations')
axes[0].set_ylabel('Probability')

# For Random Forest
sorted_idx_rf = np.argsort(rf_probs)
axes[1].plot(rf_probs[sorted_idx_rf], color='blue')
axes[1].scatter(range(len(y_test)), y_test.values[sorted_idx_rf], color='red', s=10)
axes[1].set_title('Random Forest')
axes[1].set_xlabel('Sorted Observations')

# For SVM
sorted_idx_svm = np.argsort(svm_probs)
axes[2].plot(svm_probs[sorted_idx_svm], color='blue')
axes[2].scatter(range(len(y_test)), y_test.values[sorted_idx_svm], color='red', s=10)
axes[2].set_title('SVM')
axes[2].set_xlabel('Sorted Observations')


plt.legend(loc='lower right')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig("predicted_probabilities_all_models.png", dpi=300)
plt.show()


In [ ]:
#Moderation analysis to check the moderation effect between two indipendent variables on dependent variables
df = df.rename(columns={
    'Fuelwood Collection': 'Fuelwood_Collection',
    'Inadequate Education': 'Inadequate_Education',
    'Deforestation': 'Deforestation'
})
df['Interaction'] = df['Fuelwood_Collection'] * df['Inadequate_Education']

model = smf.logit("Deforestation ~ Fuelwood_Collection + Inadequate_Education + Interaction", data=df).fit()
print(model.summary())


# Creating a new DataFrame for prediction
plot_data = pd.DataFrame({
    'Fuelwood_Collection': np.tile(np.linspace(0, 1, 100), 2),
    'Inadequate_Education': np.repeat([0, 1], 100)
})
plot_data['Interaction'] = plot_data['Fuelwood_Collection'] * plot_data['Inadequate_Education']
plot_data['predicted_prob'] = model.predict(plot_data)

# Plotting the moderation analysis
plt.figure(figsize=(8, 6))
sns.lineplot(
    x='Fuelwood_Collection',
    y='predicted_prob',
    hue='Inadequate_Education',
    palette='Set1',
    data=plot_data
)
plt.title('Moderation Effect of Education on Fuelwood Collection and Deforestation')
plt.xlabel('Fuelwood Collection (0=No, 1=Yes)')
plt.ylabel('Predicted Probability of Deforestation')
plt.legend(title='Inadequate Education')
plt.grid(True)
plt.tight_layout()
plt.savefig("moderation_plot.png", dpi=300)
plt.show()


In [ ]:
#Creating heat maps of land distribution across the respondents 
# Selecting the land use columns
land_cols = ['Agriculture', 'Forest', 'Rangeland', 'Barren land']
land_data = df[land_cols]

# Creating the correlation matrix
corr_matrix = land_data.corr()

# Rows are respondents and columns are land use types
plt.figure(figsize=(6, 5))
sns.heatmap(land_data, cmap='viridis', cbar=True, xticklabels=True, yticklabels=False)
plt.title('Land Use Distribution Across Respondents')
plt.xlabel('Land Use Type')
plt.ylabel('Respondents')
plt.tight_layout()
plt.savefig("land_use_distribution_heatmap.png", dpi=500)
plt.show()



In [ ]:
#Creating heat maps of land use types across the respondents 
plt.figure(figsize=(6, 5))
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu', linewidths=0.5)
plt.title('Correlation Heatmap of Land Use Types')
plt.tight_layout()
plt.savefig("land_use_correlation_heatmap.png", dpi=300)
plt.show()


In [ ]:
##Creating heat maps of key deforestation drivers 
numeric_cols = ['Population Growth', 'Livelihood activities','Forest Fire',	'Fuelwood Collection', 'Inadequate Education', 'Poor Forest Management',
        'Illegal Logging',	'Distance from Forest']

# Computing the correlation
corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", square=True,
            cbar_kws={'shrink': 0.8}, annot_kws={"size": 10})

plt.title("Correlation Heatmap of Key Deforestation Predictors", fontsize=14)
plt.tight_layout()

# Saving image (300 DPI)
plt.savefig("figure3_correlation_heatmap.png", dpi=500)
plt.show()